In [1]:
import requests
from urllib.error import URLError, HTTPError
from http.client import IncompleteRead
from time import sleep

In [2]:
# kakao api
APP_KEY = "50ec8e7b1c4b4a0cb78c5ada1ebff80b"
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

In [3]:
def kakao_req(address):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)
    kakao_url_string = str(kakao_url)+ '?query='+ str(address)
    try:
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except HTTPError as e:
        print("502 error caused reply" + e)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except IncompleteRead:
        # Oh well, reconnect and keep trucking
        print("_chunk_size Error")
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except URLError as e:
        print(e.reason)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    else:
        data= resultXML.json()
        if len(data['documents']) ==0:
            data= '403'
            return data
        else:
            return resultXML

In [4]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = kakao_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [5]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np
from pyspark.sql import SQLContext

In [6]:
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [7]:
basic_folder = './office_rent_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)

office_rent_201701.csv
office_rent_201702.csv
office_rent_201703.csv
office_rent_201704.csv
office_rent_201705.csv
office_rent_201706.csv
office_rent_201707.csv
office_rent_201708.csv
office_rent_201709.csv
office_rent_201710.csv
office_rent_201711.csv
office_rent_201712.csv
office_rent_201801.csv
office_rent_201802.csv
office_rent_201803.csv
office_rent_201804.csv
office_rent_201805.csv
office_rent_201806.csv
office_rent_201807.csv
office_rent_201808.csv
office_rent_201809.csv
office_rent_201810.csv
office_rent_201811.csv
office_rent_201812.csv
office_rent_201901.csv
office_rent_201902.csv
office_rent_201903.csv
office_rent_201904.csv
office_rent_201905.csv
office_rent_201906.csv
office_rent_201907.csv
office_rent_201908.csv
office_rent_201909.csv
office_rent_201910.csv
office_rent_201911.csv
office_rent_201912.csv
office_rent_202001.csv
office_rent_202002.csv
office_rent_202003.csv
office_rent_202004.csv
office_rent_202005.csv
office_rent_202006.csv
office_rent_202007.csv
office_rent

In [8]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- deposit: string (nullable = true)
 |-- rental_fee: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [10]:
df.show()
df.count()

+---+----------+-------+----------+----+-----+---+----------+--------+-------+-----+----------------------+-----+----------------------+-------------------------+----------------+
|_c0|build_year|deposit|rental_fee|year|month|day|      city|sub_city|   dong| code|                  name|floor|exclusive_private_area|                  address|detailed_address|
+---+----------+-------+----------+----+-----+---+----------+--------+-------+-----+----------------------+-----+----------------------+-------------------------+----------------+
|  0|      1998|  1,000|        50|2020|    2| 26|서울특별시|  종로구| 필운동|11110|            파크뷰타워|    5|                 25.86|서울특별시 종로구  필운동|           285=5|
|  1|      1998| 15,000|         0|2020|    2| 28|서울특별시|  종로구| 필운동|11110|            파크뷰타워|    7|                 25.86|서울특별시 종로구  필운동|           285=5|
|  2|      2008|  5,000|        70|2020|    2| 15|서울특별시|  종로구| 사직동|11110| 광화문 풍림스페이스본|    5|                 39.54|서울특별시 종로구  사직동|               9|
|  3| 

449174

In [11]:
filter_df = df.na.drop()

In [12]:
from pyspark.sql.functions import *

In [13]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))
# agg = exprs에 들어있는 계산식으로 dataframe 계산

unique_date_list = unique_date_df.collect()
new_date_list = []
for unique_date in unique_date_list:
    
    if unique_date[1] < 10 :
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
        
    else:
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [14]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+---+---+----------+-------+----------+----------+---------------+-----------+-----+------------------------------+-----+----------------------+------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|deposit|rental_fee|      city|       sub_city|       dong| code|                          name|floor|exclusive_private_area|                       address|detailed_address|               date|
+----+-----+---+---+----------+-------+----------+----------+---------------+-----------+-----+------------------------------+-----+----------------------+------------------------------+----------------+-------------------+
|2017|    1|  1|  1|      2002|  2,500|         0|    경기도|         안성시|대덕면 내리|41550|                  명경오피스텔|    4|                  22.5|     경기도 안성시 대덕면 내리|           694=1|2017-01-01T00:00:00|
|2017|    1|  1|  0|      2016|    500|        45|부산광역시|         연제구|     거제동|26470|                   유림제우스B|    2|                

424174

In [15]:
added_date_df = added_date_df.withColumn("detailed_address", regexp_replace(col("detailed_address"),"=","-"))
added_date_df.show()

+----+-----+---+---+----------+-------+----------+----------+---------------+-----------+-----+------------------------------+-----+----------------------+------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|deposit|rental_fee|      city|       sub_city|       dong| code|                          name|floor|exclusive_private_area|                       address|detailed_address|               date|
+----+-----+---+---+----------+-------+----------+----------+---------------+-----------+-----+------------------------------+-----+----------------------+------------------------------+----------------+-------------------+
|2017|    1|  1|  1|      2002|  2,500|         0|    경기도|         안성시|대덕면 내리|41550|                  명경오피스텔|    4|                  22.5|     경기도 안성시 대덕면 내리|           694-1|2017-01-01T00:00:00|
|2017|    1|  1|  0|      2016|    500|        45|부산광역시|         연제구|     거제동|26470|                   유림제우스B|    2|                

In [16]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+-----------------------------+----------------+-----+
|                      address|detailed_address|count|
+-----------------------------+----------------+-----+
| 경기도 수원시 영통구  이의동|            1330|  276|
|    대전광역시 유성구  지족동|           863-2|   75|
|      경기도 남양주시  다산동|          6060-2|   36|
|      서울특별시 중구  흥인동|               8|   21|
|    부산광역시 금정구  장전동|          227-12|   12|
|    서울특별시 강북구  수유동|            48-1|   85|
| 경기도 용인시 수지구  상현동|          1133-2|  163|
|    서울특별시 강서구  마곡동|           784-4|  316|
|    서울특별시 은평구  불광동|           319-8|   13|
|    서울특별시 강동구  암사동|           502-3|   13|
|    서울특별시 관악구  봉천동|          912-14|   34|
|      서울특별시 도봉구  창동|         731-113|    8|
|    서울특별시 강동구  천호동|          167-57|   16|
|경상북도 경산시 하양읍 금락리|          161-32|    7|
|    서울특별시 금천구  독산동|          331-40|   25|
|      충청남도 서산시  읍내동|         102-125|   86|
|    서울특별시 강동구  성내동|            43-6|   35|
| 경기도 안양시 동안구  호계동|          892-15|   17|
|   경기도 용인시 기흥구  중동|          1111-7|   34|


11950

In [17]:
sum = new_df.select(sum("count"))
sum.show()

+----------+
|sum(count)|
+----------+
|    424174|
+----------+



In [18]:
unique_address_list = new_df.collect()
#print (unique_address_list[0][0])

In [19]:
import sys

def req_addr(address):
    data=0
    if '-' in address[1]:
        check = str(address[1]).split('-')[0]
        if address[1].split('-')[0].isdigit() == False or address[1].split('-')[1].isdigit() == False:
            print(address)
            print('주소에 글자섞임')
            data= 400
    if data == 0:
        response = kakao_req(address[0] + " " + address[1])
    else :
        return data
    if response == '403':
        print('빈배열 에러 ' + address[0] + " " + address[1])
        sleep(5)
        data= 3
        return data
    else :
        return response
        
    

In [20]:

location_list = []

# checking why no esult data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;
print (length)
addr2 =unique_address_list[:1]

for address in unique_address_list: 
    #full_address = (address[0] + " " + address[1]).encode('utf8')
    #print(full_address)
    response = req_addr(address)
    if response == 400 or response == 3:
        continue
    if response.status_code == 200:
        data = response.json()
        x = data['documents'][0]['address']['x']
        y = data['documents'][0]['address']['y']
        location_list.append([address[0],address[1],y,x])
    else: 
        print (response.json())
        count += 1
        #print (count)
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if index % 100 == 0:
        print (("%0.2f" % (index / length * 100.0)) + "% completed")
        
    index += 1.0

11950
0.84% completed
빈배열 에러 경기도 용인시 기흥구  구갈동 662
1.67% completed
빈배열 에러 서울특별시 동대문구  용두동 100-8
2.51% completed
3.35% completed
4.18% completed
빈배열 에러 경기도 시흥시  정왕동 1771-24
5.02% completed
5.86% completed
빈배열 에러 부산광역시 동래구  온천동 189-49
6.69% completed
7.53% completed
8.37% completed
9.21% completed
10.04% completed
10.88% completed
빈배열 에러 경상남도 양산시  평산동 184
11.72% completed
12.55% completed
13.39% completed
14.23% completed
15.06% completed
15.90% completed
16.74% completed
17.57% completed
18.41% completed
빈배열 에러 서울특별시 영등포구  영등포동2가 306-2
19.25% completed
20.08% completed
20.92% completed
21.76% completed
빈배열 에러 경기도 부천시  중동 1153
22.59% completed
23.43% completed
24.27% completed
25.10% completed
25.94% completed
26.78% completed
27.62% completed
28.45% completed
29.29% completed
30.13% completed
빈배열 에러 서울특별시 관악구  신림동 1430-23
30.96% completed
31.80% completed
32.64% completed
33.47% completed
34.31% completed
35.15% completed
35.98% completed
36.82% completed
빈배열 에러 강원도 원주시  일산동 185-1
37.66%

In [21]:
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+-----------------------------+----------------+----------------+----------------+
|                      address|detailed_address|        latitude|       longitude|
+-----------------------------+----------------+----------------+----------------+
| 경기도 수원시 영통구  이의동|            1330|37.2925283683075|127.049160071729|
|    대전광역시 유성구  지족동|           863-2|36.3825806256824|127.320554874073|
|      경기도 남양주시  다산동|          6060-2|37.6244552414091|127.151553661342|
|      서울특별시 중구  흥인동|               8|37.5677164459151|127.016349553927|
|    부산광역시 금정구  장전동|          227-12|35.2372286857415|129.087989071154|
|    서울특별시 강북구  수유동|            48-1|37.6344211076685|127.022862985412|
| 경기도 용인시 수지구  상현동|          1133-2|37.2959868892725|127.069970873404|
|    서울특별시 강서구  마곡동|           784-4| 37.565515930868|126.840040467869|
|    서울특별시 은평구  불광동|           319-8|37.6199384778402|126.924162549321|
|    서울특별시 강동구  암사동|           502-3| 37.550953080587|127.126918876322|
|    서울특별시 관악구  봉천동|          9

In [22]:
location.count()

11929

In [23]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [24]:
joined_df.show()
joined_df.count()

+-----------------------+----------------+----+-----+---+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|                address|detailed_address|year|month|day|_c0|build_year|deposit|rental_fee|  city|sub_city|   dong| code|                name|floor|exclusive_private_area|               date|        latitude|       longitude|
+-----------------------+----------------+----+-----+---+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|경기도 남양주시  다산동|          6060-2|2020|    7|  1| 26|      2020|    500|        45|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    7|                 20.62|2020-07-01T00:00:00|37.6244552414091|127.151553661342|
|경기도 남양주시  다산동|          6060-2|2020|    5| 19| 50|      2020| 12,000|         0|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    8|      

423220

In [25]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+-----------------------+----------------+----+-----+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|                address|detailed_address|year|month|day|build_year|deposit|rental_fee|  city|sub_city|   dong| code|                name|floor|exclusive_private_area|               date|        latitude|       longitude|
+-----------------------+----------------+----+-----+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|경기도 남양주시  다산동|          6060-2|2020|    7|  1|      2020|    500|        45|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    7|                 20.62|2020-07-01T00:00:00|37.6244552414091|127.151553661342|
|경기도 남양주시  다산동|          6060-2|2020|    5| 19|      2020| 12,000|         0|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    8|                 20.62|202

423220

In [26]:
final_apartment_df = drop_index_df.na.drop()
final_apartment_df = final_apartment_df.withColumn('deposit',regexp_replace('deposit',"\\,", "")).withColumn('rental_fee',regexp_replace('rental_fee',"\\,", ""))
final_apartment_df.show()
final_apartment_df.count()

+-----------------------+----------------+----+-----+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|                address|detailed_address|year|month|day|build_year|deposit|rental_fee|  city|sub_city|   dong| code|                name|floor|exclusive_private_area|               date|        latitude|       longitude|
+-----------------------+----------------+----+-----+---+----------+-------+----------+------+--------+-------+-----+--------------------+-----+----------------------+-------------------+----------------+----------------+
|경기도 남양주시  다산동|          6060-2|2020|    7|  1|      2020|    500|        45|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    7|                 20.62|2020-07-01T00:00:00|37.6244552414091|127.151553661342|
|경기도 남양주시  다산동|          6060-2|2020|    5| 19|      2020|  12000|         0|경기도|남양주시| 다산동|41360|다산 중앙하이츠 파크|    8|                 20.62|202

423220

In [27]:
#final_apartment_df.write.format('com.databricks.spark.csv').save('mycsv.csv')
final_apartment_df.coalesce(1).write.csv('office_rent.csv')